In [1]:
import pandas as pd
from spark_session import LocalSparkSession
from dataset import Dataset
from mr_id3 import MapReduceIDR3
from pyspark.mllib.tree import DecisionTree

In [2]:
from log import log
from sklearn.impute import SimpleImputer
from pyspark.sql.functions import lit
from pyspark.sql.functions import col, sum
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [3]:
%time
num_fields = [
    'age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss',
    'hours_per_week', ]

categorical_fields = [
    'workclass', 'education',
    'marital_status', 'occupation', 'relationship',
    'race', 'sex', 'native_country', ]

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.72 µs


In [4]:
%time
target = 'label'
filename = 'dataset/adult.data'

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs


In [5]:
%time
cores = [4, ]
multiply = list(range(1, 21))
metrics = []

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


In [6]:
%time
for n_cores in cores:
    for m_factor in multiply:
        spark = LocalSparkSession(n_cores)
        spark.start()

        dataset = Dataset(spark.spark, filename, num_fields, categorical_fields, target)
        dataset.load()
        dataset.one_hot_encode_categorical_fields()
        dataset.multiply_dataset(m_factor)

        mr_id3 = MapReduceIDR3(dataset)
        mr_id3.set_labeled_point()
        mr_id3.split()
        mr_id3.train()
        mr_id3.predict()
        
        metric = mr_id3.get_metrics()
        metric['length_rows'] = dataset.df.count()
        metric['m_factor'] = m_factor
        metric['n_cores'] = n_cores
        metrics.append(metric)
        print(m_factor, 'OK')

        spark.stop()

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.2 µs
22/10/15 10:11:31 WARN Utils: Your hostname, Mac-Pro-de-MARCELO.local resolves to a loopback address: 127.0.0.1; using 192.168.0.62 instead (on interface en2)
22/10/15 10:11:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/15 10:11:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/Users/marcelovasconcellos/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


1 OK


...

10 OK


11 OK


12 OK


ERROR:root:Exception while sending command.                        (4 + 4) / 14]
Traceback (most recent call last):
  File "/Users/marcelovasconcellos/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=81>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/marcelovasconcellos/opt/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/marcelovasconcellos/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/marcelovasconcellos/opt/anaconda3/lib/python3

Py4JError: An error occurred while calling o25021.areaUnderPR

In [ ]:
%time
df = pd.DataFrame.from_dict(metrics)

In [ ]:
df

In [ ]:
%time
df.plot.line('m_factor', 'time')